# Bank Mortgage Comparison Tool

Compare multiple bank mortgage offers and find the best option based on ROCE (Return on Capital Employed).

**ROCE = (Annual Rental Income - Annual Interest - Annual Fees) / Down Payment × 100**

💡 **Important:** Principal payments build equity (capital creation), so only interest and fees are considered costs when calculating ROCE.


In [4]:
import pandas as pd
import numpy as np

def calculate_monthly_payment(total_borrowed, interest_rate_annual, number_of_years):
    """Calculate monthly payment for French-style mortgage."""
    monthly_interest_rate = (interest_rate_annual / 100) / 12
    number_of_months = number_of_years * 12
    
    if monthly_interest_rate > 0:
        monthly_payment = total_borrowed * (monthly_interest_rate * (1 + monthly_interest_rate)**number_of_months) / \
                          ((1 + monthly_interest_rate)**number_of_months - 1)
    else:
        monthly_payment = total_borrowed / number_of_months
    
    return monthly_payment, number_of_months

def calculate_annual_interest(total_borrowed, monthly_payment, monthly_interest_rate):
    """Calculate annual interest paid in the first year (for ROCE calculation).
    
    Note: Only interest is a cost. Principal payments build equity (capital creation).
    """
    remaining_balance = total_borrowed
    annual_interest = 0
    
    # Calculate interest for first 12 months
    for _ in range(12):
        interest_payment = remaining_balance * monthly_interest_rate
        principal_payment = monthly_payment - interest_payment
        remaining_balance = remaining_balance - principal_payment
        annual_interest += interest_payment
    
    return annual_interest

def calculate_total_interest(total_borrowed, monthly_payment, number_of_months, monthly_interest_rate):
    """Calculate total interest paid over the loan term."""
    remaining_balance = total_borrowed
    total_interest_paid = 0
    
    for _ in range(1, number_of_months + 1):
        interest_payment = remaining_balance * monthly_interest_rate
        principal_payment = monthly_payment - interest_payment
        remaining_balance = remaining_balance - principal_payment
        total_interest_paid += interest_payment
    
    return total_interest_paid

def calculate_roce(annual_rental_income, annual_interest_cost, annual_fees, down_payment):
    """Calculate Return on Capital Employed (ROCE).
    
    Note: Only interest and fees are costs. Principal payments build equity (capital creation).
    ROCE = (Annual Rental Income - Annual Interest - Annual Fees) / Down Payment × 100
    """
    annual_net_income = annual_rental_income - annual_interest_cost - annual_fees
    if down_payment > 0:
        roce = (annual_net_income / down_payment) * 100
    else:
        roce = float('inf') if annual_net_income > 0 else float('-inf')
    return roce, annual_net_income


## Input: Bank Offers & Property Details

Enter the property details and list of bank offers. Each bank offer should include:
- **bank_name**: Name of the bank
- **interest_rate_annual**: Annual interest rate (%)
- **number_of_years**: Loan term in years
- **loan_amount**: Amount to borrow
- **monthly_fees**: Combined monthly fees (insurance, cleaning, taxes, etc.)
- **down_payment**: Down payment required

Also specify:
- **property_price**: Total property price
- **monthly_rental_income**: Expected monthly rental income


In [ ]:
# ===== PROPERTY DETAILS =====
property_price = 106_650  # Total property price
monthly_rental_income = 675  # Expected monthly rental income
# ============================

# ===== BANK OFFERS =====
# Each dictionary represents one bank offer
bank_offers = [
    {
        'bank_name': 'Bank A',
        'interest_rate_annual': 3.5,
        'number_of_years': 25,
        'loan_amount': 74_200,
        'monthly_fees': 150,  # Combined: insurance + cleaning + taxes
        'down_payment': 32_450  # property_price - loan_amount
    },
    {
        'bank_name': 'Bank B',
        'interest_rate_annual': 2.95,
        'number_of_years': 25,
        'loan_amount': 74_200,
        'monthly_fees': 180,
        'down_payment': 32_450
    },
    {
        'bank_name': 'Bank C',
        'interest_rate_annual': 3.2,
        'number_of_years': 30,
        'loan_amount': 74_200,
        'monthly_fees': 140,
        'down_payment': 32_450
    }
]
# ============================


## Calculate & Compare

Run the cell below to calculate ROCE for each bank and see which is the best option.


In [ ]:
# Calculate metrics for each bank
comparison_data = []
annual_rental_income = monthly_rental_income * 12

for offer in bank_offers:
    # Calculate monthly mortgage payment
    monthly_payment, number_of_months = calculate_monthly_payment(
        offer['loan_amount'],
        offer['interest_rate_annual'],
        offer['number_of_years']
    )
    
    # Calculate annual interest (first year) - this is the actual cost
    monthly_interest_rate = (offer['interest_rate_annual'] / 100) / 12
    annual_interest = calculate_annual_interest(
        offer['loan_amount'],
        monthly_payment,
        monthly_interest_rate
    )
    
    # Calculate annual fees
    annual_fees = offer['monthly_fees'] * 12
    
    # Calculate annual principal payment (equity building)
    annual_principal = (monthly_payment * 12) - annual_interest
    
    # Calculate ROCE (only interest and fees are costs)
    roce, annual_net_income = calculate_roce(
        annual_rental_income,
        annual_interest,
        annual_fees,
        offer['down_payment']
    )
    
    # Calculate total interest over loan term
    total_interest = calculate_total_interest(
        offer['loan_amount'],
        monthly_payment,
        number_of_months,
        monthly_interest_rate
    )
    
    comparison_data.append({
        'Bank': offer['bank_name'],
        'Interest Rate (%)': offer['interest_rate_annual'],
        'Loan Term (years)': offer['number_of_years'],
        'Loan Amount': offer['loan_amount'],
        'Down Payment': offer['down_payment'],
        'Monthly Payment': round(monthly_payment, 2),
        'Monthly Fees': offer['monthly_fees'],
        'Annual Interest': round(annual_interest, 2),
        'Annual Principal': round(annual_principal, 2),
        'Annual Fees': round(annual_fees, 2),
        'Annual Net Income': round(annual_net_income, 2),
        'ROCE (%)': round(roce, 2),
        'Total Interest (lifetime)': round(total_interest, 2)
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)

# Sort by ROCE (descending) to show best option first
comparison_df = comparison_df.sort_values('ROCE (%)', ascending=False).reset_index(drop=True)

# Display results
print("=" * 100)
print("BANK MORTGAGE COMPARISON - ROCE ANALYSIS")
print("=" * 100)
print(f"\nProperty Price: ${property_price:,.2f}")
print(f"Monthly Rental Income: ${monthly_rental_income:,.2f}")
print(f"Annual Rental Income: ${annual_rental_income:,.2f}")
print("\n💡 Note: Principal payments build equity (capital creation), only interest and fees are costs.")
print("\n" + "=" * 100)
print()

# Display comparison table
comparison_df

# Show best option
best_bank = comparison_df.iloc[0]
print("\n" + "=" * 100)
print("🏆 BEST OPTION (Highest ROCE)")
print("=" * 100)
print(f"Bank: {best_bank['Bank']}")
print(f"ROCE: {best_bank['ROCE (%)']:.2f}%")
print(f"Annual Net Income: ${best_bank['Annual Net Income']:,.2f}")
print(f"Monthly Payment: ${best_bank['Monthly Payment']:,.2f}")
print(f"Annual Interest (cost): ${best_bank['Annual Interest']:,.2f}")
print(f"Annual Principal (equity): ${best_bank['Annual Principal']:,.2f}")
print(f"Monthly Fees: ${best_bank['Monthly Fees']:,.2f}")
print(f"Interest Rate: {best_bank['Interest Rate (%)']:.2f}%")
print(f"Loan Term: {best_bank['Loan Term (years)']} years")
print("=" * 100)


BANK MORTGAGE COMPARISON - ROCE ANALYSIS

Property Price: $106,650.00
Monthly Rental Income: $800.00
Annual Rental Income: $9,600.00



🏆 BEST OPTION (Highest ROCE)
Bank: Bank C
ROCE: 12.54%
Annual Net Income: $4,069.32
Monthly Payment: $320.89
Monthly Fees: $140.00
Total Monthly Cost: $460.89
Interest Rate: 3.20%
Loan Term: 30 years
